In [1]:

%load_ext autoreload
%autoreload 2

# word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

## Training

Download some data, for example: [http://mattmahoney.net/dc/text8.zip](http://mattmahoney.net/dc/text8.zip)

In [2]:

import word2vec

Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

In [4]:

word2vec.word2phrase('./data/text8', './data/text8-phrases', verbose=False)

This will create a `text8-phrases` that we can use as a better input for `word2vec`.
Note that you could easily skip this previous step and use the origial data as input for `word2vec`.

Train the model using the `word2phrase` output.

In [6]:

word2vec.word2vec('./data/text8-phrases', './data/text8.bin', size=100, verbose=False)

That generated a `text8.bin` file containing the word vectors in a binary format.

Do the clustering of the vectors based on the trained model.

In [7]:

word2vec.word2clusters('./data/text8', './data/text8-clusters.txt', 100, verbose=False)

That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [8]:

import word2vec

Import the `word2vec` binary file created above

In [9]:

model = word2vec.load('./data/text8.bin')

We can take a look at the vocabulaty as a numpy array

In [10]:

model.vocab

array(['</s>', 'the', 'of', ..., 'dupree', 'rauchbier', 'erythropoietin'], 
      dtype='<U78')

Or take a look at the whole matrix

In [11]:

model.vectors.shape

(98331, 100)

In [12]:

model.vectors

array([[-0.16299245, -0.12382638, -0.11257624, ...,  0.08034179,
        -0.12345736, -0.11461084],
       [-0.18662284, -0.02103809, -0.0642486 , ...,  0.01556384,
        -0.02935963, -0.11644705],
       [-0.07731996, -0.04529019, -0.07655367, ..., -0.0069496 ,
         0.10263547,  0.06946383],
       ..., 
       [-0.00223393, -0.07478746, -0.14914325, ..., -0.07335386,
        -0.0908418 ,  0.14270267],
       [ 0.03041174,  0.07300348, -0.2029229 , ..., -0.13777271,
         0.0160792 ,  0.05938902],
       [-0.16532739,  0.01784582, -0.28873158, ..., -0.06535831,
        -0.01124263,  0.11558969]])

We can retreive the vector of individual words

In [13]:

model['dog'].shape

(100,)

In [14]:

model['dog'][:10]

array([-0.11056666, -0.00713978, -0.12022977, -0.04947436,  0.00867581,
        0.01924019, -0.032873  , -0.09843912,  0.17465644,  0.16547856])

We can do simple queries to retreive words similar to "socks" based on cosine similarity:

In [15]:

indexes, metrics = model.cosine('socks')
indexes, metrics

(array([19940, 10195, 25219, 14620,  6055, 72400,  4311, 20496,  9504, 12054], dtype=int64),
 array([ 0.76899715,  0.75779345,  0.75251087,  0.74938868,  0.74755605,
         0.74265468,  0.74083339,  0.74059521,  0.74022767,  0.73507522]))

This returned a tuple with 2 items:
1. numpy array with the indexes of the similar words in the vocabulary
2. numpy array with cosine similarity to each word

Its possible to get the words of those indexes

In [16]:

model.vocab[indexes]

array(['hairy', 'pitched', 'glove', 'winged', 'hat', 'bellied', 'oh',
       'slender', 'candy', 'jacket'], 
      dtype='<U78')

There is a helper function to create a combined response: a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [17]:

model.generate_response(indexes, metrics)

rec.array([('hairy',  0.76899715), ('pitched',  0.75779345), ('glove',  0.75251087),
 ('winged',  0.74938868), ('hat',  0.74755605), ('bellied',  0.74265468),
 ('oh',  0.74083339), ('slender',  0.74059521), ('candy',  0.74022767),
 ('jacket',  0.73507522)], 
          dtype=[('word', '<U78'), ('metric', '<f8')])

Is easy to make that numpy array a pure python response:

In [18]:

model.generate_response(indexes, metrics).tolist()

[('hairy', 0.76899714547413),
 ('pitched', 0.7577934499914786),
 ('glove', 0.7525108722361309),
 ('winged', 0.7493886797915918),
 ('hat', 0.7475560474528551),
 ('bellied', 0.7426546824203596),
 ('oh', 0.7408333893914528),
 ('slender', 0.7405952082289993),
 ('candy', 0.7402276658082224),
 ('jacket', 0.7350752212195315)]

### Phrases

Since we trained the model with the output of `word2phrase` we can ask for similarity of "phrases"

In [19]:

indexes, metrics = model.cosine('los_angeles')
model.generate_response(indexes, metrics).tolist()

[('san_francisco', 0.8513235414458705),
 ('detroit', 0.8227418019557884),
 ('california', 0.8150192231189138),
 ('cincinnati', 0.8042854970065477),
 ('boston', 0.7990556118308071),
 ('chicago', 0.7925396462502889),
 ('cleveland', 0.7915046846773899),
 ('new_jersey', 0.78379894443633),
 ('atlanta', 0.7759746807818365),
 ('melbourne', 0.7712548324159106)]

### Analogies

Its possible to do more complex queries like analogies such as: `king - man + woman = queen` 
This method returns the same as `cosine` the indexes of the words in the vocab and the metric

In [20]:

indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'], n=10)
indexes, metrics

(array([  648,  7530,  6756,  1145,  1006,   344,  2818,  3219, 32851,  1770], dtype=int64),
 array([ 0.31314374,  0.29866589,  0.29340463,  0.29305534,  0.29270227,
         0.29046997,  0.28714982,  0.28410298,  0.2839819 ,  0.28214124]))

In [21]:

model.generate_response(indexes, metrics).tolist()

[('emperor', 0.3131437388886312),
 ('empress', 0.2986658934165427),
 ('roman_emperor', 0.29340462805072975),
 ('prince', 0.293055335459291),
 ('daughter', 0.29270226826611656),
 ('son', 0.29046997096006294),
 ('ruler', 0.28714981729001143),
 ('archbishop', 0.2841029831363603),
 ('khagan', 0.2839819038287986),
 ('pope', 0.2821412429986396)]

In [24]:

indexes, metrics = model.analogy(pos=['fire', 'belly'], neg=['tongue'], n=10)
model.generate_response(indexes, metrics).tolist()

[('storm', 0.27951571176717627),
 ('flood', 0.26762447520406474),
 ('suddenly', 0.2643262148023257),
 ('drowning', 0.26123349099657167),
 ('strikes', 0.2581002738073477),
 ('suffocation', 0.2552033337520781),
 ('balloon', 0.2546410908948632),
 ('panicked', 0.25108227139003075),
 ('gunman', 0.24925652966151818),
 ('crash', 0.24871098496612656)]

### Clusters

In [38]:

clusters = word2vec.load_clusters('./data/text8-clusters.txt')
clusters.vocab

array([b'</s>', b'the', b'of', ..., b'koba', b'skirting', b'selectors'], dtype=object)

We can see get the cluster number for individual words

In [42]:

clusters.ix(b'skirting')

71289

We can see get all the words grouped on an specific cluster

In [44]:

clusters.get_words_on_cluster(90).shape

(513,)

In [45]:

clusters.get_words_on_cluster(90)[:10]

array([b'had', b'became', b'did', b'said', b'took', b'wrote', b'went',
       b'gave', b'saw', b'appeared'], dtype=object)

We can add the clusters to the word2vec model and generate a response that includes the clusters

In [46]:

model.clusters = clusters

In [47]:

indexes, metrics = model.analogy(pos=['paris', 'germany'], neg=['france'], n=10)

In [48]:

model.generate_response(indexes, metrics).tolist()

[('leipzig', 0.2875431924348552, 96),
 ('vienna', 0.2795215766150148, 85),
 ('munich', 0.2793942749830143, 53),
 ('berlin', 0.27549881085272704, 47),
 ('grammar_school', 0.2685379878357954, 8),
 ('venice', 0.26594170611087065, 39),
 ('milan', 0.26252236256033934, 14),
 ('mainz', 0.26228894654995816, 13),
 ('basel', 0.26192267355304044, 64),
 ('lillehammer', 0.2592898738060045, 90)]